In [1]:
import folium
import datetime as dt
import random as rnd
import csv
import geopandas
import polyline
import requests

In [13]:
import json

coord_pairs = []

with open('data/population_from_VEHITS_sams_od.json') as json_file:
    data = json.load(json_file)
    for person in data['persons']:
        start_coord = person['activities'][0]['coord']
        end_coord = person['activities'][1]['coord']
        time = person['activities'][1]['start_time']
        szone = person['activities'][0]['zone']
        ezone = person['activities'][1]['zone']
        coord_pairs.append((start_coord['lat'],start_coord['lon'], end_coord['lat'], end_coord['lon'], time, szone, ezone))

In [14]:
def swap_xy(geom):
    if geom.is_empty:
        return geom

    if geom.has_z:
        def swap_xy_coords(coords):
            for x, y, z in coords:
                yield (y, x, z)
    else:
        def swap_xy_coords(coords):
            for x, y in coords:
                yield (y, x)

    # Process coordinates from each supported geometry type
    if geom.type in ('Point', 'LineString', 'LinearRing'):
        return type(geom)(list(swap_xy_coords(geom.coords)))
    elif geom.type == 'Polygon':
        ring = geom.exterior
        shell = type(ring)(list(swap_xy_coords(ring.coords)))
        holes = list(geom.interiors)
        for pos, ring in enumerate(holes):
            holes[pos] = type(ring)(list(swap_xy_coords(ring.coords)))
        return type(geom)(shell, holes)
    elif geom.type.startswith('Multi') or geom.type == 'GeometryCollection':
        # Recursive call
        return type(geom)([swap_xy(part) for part in geom.geoms])
    else:
        raise ValueError('Type %r not recognized' % geom.type)
        
from matplotlib import cm

def pseudocolor(val, minval, maxval):
    # Scale val to be in the range [0, 1]
    val = (val - minval) / (maxval - minval)
    # Return RGBA tuple from jet colormap
    return [int(i*255) for i in cm.gray(val)][0:3]

In [15]:
import pandas as pd
df = pd.DataFrame(coord_pairs, columns=['slat','slon','elat','elon','time', 'szone','ezone'])
# df['latlon'] = df.lat.astype(str)+df.lon.astype(str)

In [32]:
df

,slat,slon,elat,elon,time,szone,ezone
0,55.654863,13.220201,55.637863,13.710486,08:25:12,12300003,12650003
1,55.646732,13.217800,55.638653,13.726402,06:35:32,12300003,12650003
2,55.653951,13.215827,55.628642,13.697939,04:07:35,12300003,12650004
3,55.646094,13.220931,55.631606,13.697883,07:10:07,12300003,12650004
4,55.645554,13.216492,55.639343,13.685163,12:49:15,12300003,12650005
...,...,...,...,...,...,...,...
8661,55.675124,14.144037,55.719426,13.811031,06:57:20,12910027,12650013
8662,55.684185,14.206187,55.723258,13.798935,12:14:56,12910027,12650013
8663,55.700508,14.205177,55.752216,13.773824,07:32:10,12910027,12650013
8664,55.686839,14.154055,55.751798,13.836889,06:56:53,12910027,12650013


In [17]:
class OtpMode(object):
    CAR = 'CAR'
    WALK = 'WALK'
    TRANSIT = 'TRANSIT,WALK'
    BUS = 'BUS,WALK'
    RAIL = 'TRAM,RAIL,SUBWAY,FUNICULAR,GONDOLA,WALK'
    BICYCLE = 'BICYCLE'
    BICYCLE_TRANSIT = 'TRANSIT,BICYCLE'
    PARK_RIDE = 'CAR_PARK,WALK,TRANSIT'
    KISS_RIDE = 'CAR,WALK,TRANSIT'
    BIKE_RIDE = 'BICYCLE_PARK,WALK,TRANSIT'
    RENTED_BICYCLE = 'WALK,BICYCLE_RENT'
    TRANSIT_RENTED_BICYCLE = 'TRANSIT,WALK,BICYCLE_RENT'
    DRT = 'DRT'
    DRT_TRANSIT = 'DRT_TRANSIT'

    _DICT = ['CAR', 'WALK', 'TRANSIT', 'BUS', 'RAIL', 'BICYCLE', 'BICYCLE_TRANSIT', 'PARK_RIDE', 'KISS_RIDE',
             'BIKE_RIDE', 'RENTED_BICYCLE', 'TRANSIT_RENTED_BICYCLE', 'DRT', 'DRT_TRANSIT']

    _MAIN_MODES = ['CAR', 'BICYCLE', 'TRANSIT', 'WALK']

    _DRT_MODES = ['DRT', 'DRT_TRANSIT']

    _PT_MODES = ['TRANSIT', 'BUS', 'RAIL']

    @staticmethod
    def get_all_modes():
        return [OtpMode.__dict__.get(item) for item in OtpMode._DICT]

    @staticmethod
    def get_main_modes():
        return [OtpMode.__dict__.get(item) for item in OtpMode._MAIN_MODES]

    @staticmethod
    def contains(other):
        return other in OtpMode._DICT

    @staticmethod
    def get_pt_modes():
        return OtpMode._PT_MODES

    @staticmethod
    def get_drt_modes():
        return OtpMode._DRT_MODES

    @staticmethod
    def get_mode(string):
        if OtpMode.contains(string):
            return OtpMode.__getattribute__(OtpMode(), string)
        else:
            raise Exception('unsupported mode {}'.format(string))


In [18]:
class Coord(object):
    """Coordinate.

    Parameters
    ----------
    lat : <float> latitude
    lon : <float> longitude
    latlon : <list> list with both lat and long. Latitude first!
    """
    def __init__(self, lat=None, lon=None, latlon=None):
        if latlon is not None:
            if len(latlon) != 2:
                raise Exception("Wrong coordinate latlon format. Should be a list of two floats.")
            self.lat = latlon[0]
            self.lon = latlon[1]
        elif lat is None or lon is None:
            raise Exception("Coordinates not provided")
        else:
            self.lat = lat
            self.lon = lon

    def to_json(self):
        return json.dumps(self, default=lambda o: self._try(o), sort_keys=True, indent=4, separators=(',', ':'))

    @staticmethod
    def _try(o):
        try:
            if o.__class__ == Coord:
                raise Exception()
            return o.__dict__
        except:
            return str(o)
        
    def __str__(self):
        return str(self.lat) + ',' + str(self.lon)

    def __repr__(self):
        return self.__str__()

    def __eq__(self, other):
        return self.lat == other.lat and self.lon == other.lon

    def __hash__(self):
        return hash((self.lat, self.lon))


In [19]:
class Leg(object):
    """Leg of a trip. For example, "walk - bus - walk" trip has three legs.
    Used to store trip legs from OTP.

    Parameters
    ----------
    mode : <str> mode of transport
    start_coord : <coord> coordinate of an origin
    end_coord : <coord> coordinate of a destination
    distance : <int> meters
    duration : <int> seconds
    steps : <list> of utils.Step
    """

    # TODO:assignment of mode   as a string is confusing, remove it, or use constant
    def __init__(self, mode=None, start_coord=None, from_stop=None, end_coord=None, to_stop=None,
                 start_time=None, end_time=None,
                 distance=None, duration=None, steps=None):
        self.mode = mode
        self.start_coord = start_coord
        self.end_coord = end_coord
        self.distance = distance
        self.duration = duration
        self.steps = steps
        # The two below only used for PT legs
        self.from_stop = from_stop
        self.to_stop = to_stop

        self.start_time = start_time
        self.end_time = end_time

    def deepcopy(self):
        if self.steps is None:
            steps = []
        else:
            steps = [step.deepcopy() for step in self.steps if step is not None]
        return Leg(mode=copy.copy(self.mode),
                   start_coord=copy.copy(self.start_coord),
                   from_stop=copy.copy(self.from_stop),
                   end_coord=copy.copy(self.end_coord),
                   to_stop=copy.copy(self.to_stop),
                   start_time=copy.copy(self.start_time),
                   end_time=copy.copy(self.end_time),
                   distance=copy.copy(self.distance),
                   duration=copy.copy(self.duration),
                   steps=steps)


In [20]:
class Trip(object):
    """A list of legs and a total trip duration
    """
    legs = ...  # type: List[Leg]

    def __init__(self):
        self.legs = []
        self.duration = None
        self.distance = None
        self.main_mode = None

    def set_empty_trip(self, mode, coord_start, coord_end):
        """Sets a dummy trip between two coordinates with zero distance, duration and one empty leg"""
        self.set_duration(0)
        self.set_distance(0)
        self.main_mode = mode
        self.legs = [Leg(mode=mode, start_coord=coord_start, end_coord=coord_end, distance=0, duration=0,
                         steps=[Step(coord_start, coord_end, 0, 0)])]

    def dumps(self):
        return self.__dict__

    def get_leg_modes(self):
        """Returns a list of modes from the legs"""
        return [l.mode for l in self.legs]

    def deepcopy(self):
        nt = Trip()
        nt.duration = copy.copy(self.duration)
        nt.distance = copy.copy(self.distance)
        nt.main_mode = copy.copy(self.main_mode)
        nt.legs = [leg.deepcopy() for leg in self.legs]
        return nt

    def main_mode_from_legs(self):
        leg_modes = self.get_leg_modes()

        if LegMode.CAR in leg_modes:
            return OtpMode.CAR
        elif LegMode.BUS in leg_modes or LegMode.SUBWAY in leg_modes or \
                LegMode.TRAM in leg_modes or LegMode.RAIL in leg_modes:
            return OtpMode.TRANSIT
        elif LegMode.BICYCLE in leg_modes:
            return OtpMode.BICYCLE
        elif LegMode.WALK in leg_modes:
            return OtpMode.BICYCLE
        else:
            log.error('Main mode unrecognized. Returning None. Kick the developer to make a proper function.')
            return None

    def set_duration(self, dur):
        self.duration = dur

    def set_main_mode(self, mode):
        self.main_mode = mode
    
    def set_distance(self, dist):
        self.distance = dist
    
    def append_leg(self, leg):
        self.legs.append(leg)
        
    def __str__(self):
        return '{} trip, takes {} distance {}'\
            .format(self.main_mode, self.duration, self.distance)

    def __repr__(self):
        return str(self)


In [21]:
class LegMode(object):
    CAR = 'CAR'
    WALK = 'WALK'
    BUS = 'BUS'
    RAIL = 'RAIL'
    BICYCLE = 'BICYCLE'
    TRAM = 'TRAM'
    SUBWAY = 'SUBWAY'
    CAR_PARK = 'CAR_PARK'
    BICYCLE_PARK = 'BICYCLE_PARK'
    BICYCLE_RENT = 'BICYCLE_RENT'
    DRT = 'DRT'

    _DICT = ['CAR', 'WALK', 'BUS', 'RAIL', 'BICYCLE', 'TRAM', 'SUBWAY',
             'CAR_PARK', 'BICYCLE_PARK', 'BICYCLE_RENT', 'DRT']

    _MAIN_MODES = ['CAR', 'BICYCLE', 'BUS', 'RAIL', 'WALK']

    @staticmethod
    def get_all_modes():
        return [LegMode.__dict__.get(item) for item in LegMode._DICT]

    @staticmethod
    def get_main_modes():
        return [LegMode.__dict__.get(item) for item in LegMode._MAIN_MODES]

    @staticmethod
    def contains(other):
        return other in LegMode._DICT

    @staticmethod
    def get_mode(string):
        if LegMode.contains(string):
            return LegMode
        else:
            raise Exception('unsupported mode {}'.format(string))

In [22]:
class Step(object):
    """Arguments:|
    start_coord       <Coord>|
    distance    <int>|
    duration    <int>|
    """
    def __init__(self, start_coord, end_coord, distance, duration):
        self.start_coord = start_coord
        self.end_coord = end_coord
        self.distance = distance
        self.duration = duration

    @staticmethod
    def get_empty_step(coord):
        return Step(start_coord=coord, end_coord=coord, distance=0, duration=0)

    def deepcopy(self):
        return Step(start_coord=copy.copy(self.start_coord),
                    end_coord=copy.copy(self.end_coord),
                    distance=copy.copy(self.distance),
                    duration=copy.copy(self.duration),
                    )

    def dumps(self):
        return self.__dict__

    def __str__(self):
        return 'Step distance {:.1f}, duration {:.1f}'.format(self.distance, self.duration)

    def __repr__(self):
        return self.__str__()


In [23]:
def parse_otp_response(resp):
    if resp.status_code != requests.codes.ok:
        resp.raise_for_status()

    jresp = resp.json()
    if 'error' in jresp.keys():
        if jresp.get('error').get('id') == 409:
            raise Exception()
        elif jresp.get('error').get('id') == 404:
            raise Exception()
        else:
            raise Exception()

    itineraries = jresp.get('plan').get('itineraries')

    trips = []
    geometries = []
    for itinerary in itineraries:
        trip = Trip()
        trip.set_duration(itinerary.get('duration'))
        trip.set_distance(sum([leg.get('distance') for leg in itinerary.get('legs')]))
        geometry = []
        for raw_leg in itinerary.get('legs'):            
            leg = Leg()
            leg.distance = raw_leg.get('distance')
            leg.duration = raw_leg.get('duration')
            raw_from = raw_leg.get('from')
            leg.start_coord = Coord(lat=raw_from.get('lat'),
                                    lon=raw_from.get('lon'))
            raw_to = raw_leg.get('to')
            leg.end_coord = Coord(lat=raw_to.get('lat'),
                                  lon=raw_to.get('lon'))
            leg.mode = raw_leg.get('mode')
#             leg.steps = [self.step_from_raw(s) for s in raw_leg.get('steps')]

#             leg.start_time = int(raw_leg.get('startTime'))/1000 - self.env.config.get('date.unix_epoch')
#             leg.end_time = int(raw_leg.get('endTime'))/1000 - self.env.config.get('date.unix_epoch')

            if leg.mode in OtpMode.get_pt_modes():
                # OTP has id in the following format: 'SE-st:9022012065015001'
                # we are not interested in the first part
                leg.from_stop = int(raw_from.get('stopId').split(':')[1])
                leg.to_stop = int(raw_to.get('stopId').split(':')[1])
            trip.append_leg(leg)

            trip.main_mode = trip.main_mode_from_legs()       
            geometry.append(raw_leg.get('legGeometry').get('points'))
        geometries.append(geometry)
        trips.append(trip)
        break

    return trips, geometries

def otp_request( from_place,
                 to_place,
                 at_time,
                 mode: str,
                 attributes=None):
    default_attributes = {'fromPlace': str(from_place),
                          'toPlace': str(to_place),
                          'date': '11-14-2018',
                          'mode': mode,
                          'arriveBy': 'True',
                          'maxWalkDistance': 2000}
    if attributes is not None:
        default_attributes.update(attributes)
    
    try:
        resp = requests.get('http://localhost:8080/otp/routers/skane/plan', params=default_attributes)
    
        # payload = Payload(attributes=default_attributes, config=self.env.config)

        # resp = requests.get(self.url, params=payload.get_payload())

        parsed_trips, geometries = parse_otp_response(resp)
    except:
        return None, None

    for trip in parsed_trips:
        trip.set_main_mode(mode)

    return parsed_trips, geometries

In [35]:
df['szone']=df.szone.astype(int)
df['ezone']=df.ezone.astype(int)

In [36]:
df.head()

,slat,slon,elat,elon,time,szone,ezone
0,55.654863,13.220201,55.637863,13.710486,08:25:12,12300003,12650003
1,55.646732,13.217800,55.638653,13.726402,06:35:32,12300003,12650003
2,55.653951,13.215827,55.628642,13.697939,04:07:35,12300003,12650004
3,55.646094,13.220931,55.631606,13.697883,07:10:07,12300003,12650004
4,55.645554,13.216492,55.639343,13.685163,12:49:15,12300003,12650005


In [42]:
polylines = []
for _,trip in df.iterrows():
    if (trip.szone < 12650000 or trip.szone > 12659999) and\
       (trip.ezone < 12650000 or trip.ezone > 12659999):
        continue
        
    trip_pt, pol_pt = otp_request(from_place='{},{}'.format(trip.slat,trip.slon),
                            to_place='{},{}'.format(trip.elat,trip.elon),
                            at_time = trip.time, mode = 'TRANSIT,WALK')
    
    trip_car, pol_car = otp_request(from_place='{},{}'.format(trip.slat,trip.slon),
                            to_place='{},{}'.format(trip.elat,trip.elon),
                            at_time = trip.time, mode = 'CAR')
    if pol_pt is not None and pol_car is not None:
        if trip_car[0].duration * 1.7 > trip_pt[0].duration:
            polylines.append(pol_pt[0])
            print(len(polylines))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [45]:
from time import gmtime, strftime

COORD_BOX = [55.63135, 13.70622]

map_skane = folium.Map(location=COORD_BOX, zoom_start=12, width=1024, height=768)

for p1 in polylines:
    for p in p1:
        leg = polyline.decode(p)
        folium.PolyLine(leg).add_to(map_skane)

In [46]:
map_skane.save('data/test.html')

In [27]:
display(map_skane)

In [ ]:
polylines = []
for _,trip in df.iterrows():
    if (trip.szone < 12650000 or trip.szone > 12659999) and\
       (trip.ezone < 12650000 or trip.ezone > 12659999):
        continue
        
    trip_pt, pol_pt = otp_request(from_place='{},{}'.format(trip.slat,trip.slon),
                            to_place='{},{}'.format(trip.elat,trip.elon),
                            at_time = trip.time, mode = 'TRANSIT,WALK')
    
    trip_car, pol_car = otp_request(from_place='{},{}'.format(trip.slat,trip.slon),
                            to_place='{},{}'.format(trip.elat,trip.elon),
                            at_time = trip.time, mode = 'CAR')
    if pol_pt is not None and pol_car is not None:
        if trip_car[0].duration * 1.5 > trip_pt[0].duration:
            polylines.append(pol_pt[0])
            print(len(polylines))